In [94]:
import sympy as sp
import numpy as np
from sympy import Matrix

In [95]:
C = sp.Matrix([
    [1, 2],
    [sp.Rational(1,2), 1]
])

A1 = sp.Matrix([
    [1, 3, sp.Rational(1,2)],
    [sp.Rational(1,3), 1, 4],
    [2, sp.Rational(1,4), 1]
])
A2 = sp.Matrix([
    [1, sp.Rational(1, 4), 2],
    [4, 1, sp.Rational(1,5)],
    [sp.Rational(1, 2), 5, 1]
])

A = [A1, A2]

In [96]:
def max_algebra_multiply_symbolic(A, B, sym_val_dict):
    rows_A, cols_A = A.shape
    rows_B, cols_B = B.shape
    if cols_A != rows_B:
        raise ValueError("Matrices are not compatible for multiplication")
    C = Matrix.zeros(rows_A, cols_B)
    
    for i in range(rows_A):
        for j in range(cols_B):
            max_val = -float('inf')
            max_expression = None
            for k in range(cols_A):
                cur_val = (A[i,k] * B[k,j]).subs(sym_val_dict)
                    
                if cur_val > max_val:
                    max_val = cur_val
                    max_expression = A[i,k] * B[k,j]

            C[i,j] = max_expression

    return C

In [97]:
def max_matrix_power_symbolic(C, k, sym_val_dict):
    if k == 0:
        return Matrix.eye(C.shape[0])
    elif k == 1:
        return C
    else:
        result = C
        for _ in range(k - 1):
            result = max_algebra_multiply_symbolic(result, C, sym_val_dict)
        return result

In [98]:
def max_algebra_addition_symbolic(matrices, sym_val_dict): 
    
    shape = matrices[0].shape
    result = Matrix.zeros(shape[0], shape[1])
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            max_value = -np.inf
            for matrix in matrices:
                matrix_ij_val = matrix[i, j].subs(sym_val_dict)
                if matrix_ij_val > max_value:
                    max_value = matrix_ij_val
                    result[i, j] = matrix[i, j]
    
    return result

In [99]:
def trace_symbolic(C, sym_val_dict):
    diagonal_elements = Matrix.diagonal(C)
    max_value = -np.inf
    for el in diagonal_elements:
        el_num = el.subs(sym_val_dict)
        if el_num > max_value:
            max_value = el_num
            max_element = el
    return max_element

In [100]:
def spectral_radius_symbolic(C, sym_val_dict):
    k = C.shape[0]
    temp = C
    max_value = -np.inf
    max_expression = None
    for i in range(1, k + 1):
        tr = trace_symbolic(temp, sym_val_dict) ** sp.Rational(1, i)
        num_tr = tr.subs(sym_val_dict)
        temp = max_algebra_multiply_symbolic(temp, C, sym_val_dict)
        if num_tr > max_value:
            max_value = num_tr
            max_expression = tr
    return max_expression

In [101]:
def klini_symbolic(C, sym_val_dict):
    r = list(sym_val_dict.keys())[-1]
    k = C.shape[0]
    tmp = Matrix.eye(k)
    matrices = [Matrix.eye(k)]
    for i in range(1, k):
        matrices.append(r**(-i)*max_algebra_multiply_symbolic(tmp, C, sym_val_dict))
        tmp = max_algebra_multiply_symbolic(tmp, C, sym_val_dict)
    return max_algebra_addition_symbolic(matrices, sym_val_dict)

In [102]:
sp_rad = spectral_radius_symbolic(C, {})
sp_rad

1

In [103]:
sp_rad_sym = sp.Symbol('lambda')
D = klini_symbolic(C, {sp_rad_sym: sp_rad})
D
#sp.latex(D)

Matrix([
[           1, 2/lambda],
[1/(2*lambda),        1]])

In [104]:
v = [
    1,
    sp.Rational(1,2)
]
v

[1, 1/2]

In [105]:
P = max_algebra_addition_symbolic([v[i]*A[i] for i in range(2)], {sp_rad_sym: sp_rad})
P

Matrix([
[1,   3, 1],
[2,   1, 4],
[2, 5/2, 1]])

In [106]:
sp_rad1 = spectral_radius_symbolic(P, {sp_rad_sym: sp_rad})
sp_rad1

sqrt(10)

In [107]:
sp_rad_sym1 = sp.Symbol('mu_1')
Q = klini_symbolic(P, {sp_rad_sym: sp_rad, sp_rad_sym1: sp_rad1})
Q
#sp.latex(Q)

Matrix([
[        1,     3/mu_1, 12/mu_1**2],
[8/mu_1**2,          1,     4/mu_1],
[   2/mu_1, 5/(2*mu_1),          1]])

In [108]:
Q_max = invert_max_columns(Q, {sp_rad_sym: sp_rad, sp_rad_sym1: sp_rad1})
Q_max

NameError: name 'invert_max_columns' is not defined

In [ ]:
def geometric_mean_rows(matrix):
    """Calculates the geometric mean of each row in a matrix.

    Args:
        matrix: A NumPy array representing the matrix.

    Returns:
        A NumPy array containing the geometric mean of each row.  Returns an array of NaNs if any row contains non-positive values.
    """
    row_products = np.prod(matrix, axis=1, keepdims=True) # keepdims to maintain 2D shape
    n = matrix.shape[1]
    return np.power(row_products, 1.0 / n)

In [ ]:
def iterative_matrix_vector_multiply_normalized(A, x, epsilon, max_iterations=1000):
    x_prev = x
    x_curr = x.copy() / np.linalg.norm(x) # Normalize the initial vector

    for i in range(max_iterations):
        norm_prev = np.linalg.norm(x_prev)
        x_prev = x_curr.copy()
        x_curr = np.dot(A, x_prev)
        norm_curr = np.sum(x_curr)
        x_curr = x_curr / norm_curr # Normalize at each iteration
        lambd = norm_curr/norm_prev
        if np.linalg.norm(x_curr - x_prev) < epsilon:
            return [x_curr, lambd, i]

    return None  # Did not converge

In [109]:
def apply_to_multiple_matrices_numpy_corrected(matrices, x, epsilon, max_iterations=1000):
    """Applies the iterative function to multiple matrices using NumPy efficiently."""
    results = []
    for matrix in matrices:
        result = iterative_matrix_vector_multiply_normalized(matrix, x, epsilon, max_iterations)
        results.append(result)
    return results #Return as a NumPy array

In [31]:
def pmatrix(a, precision=2):
    """Returns a LaTeX pmatrix with precision control

    :a: numpy array
    :precision: Number of decimal places (default is 2)
    :returns: LaTeX pmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('pmatrix can at most display two dimensions')

    lines = []
    for row in a:
        formatted_row = " & ".join([f"{x:.{precision}f}" for x in row])
        lines.append(formatted_row)


    rv = [r'\begin{pmatrix}']
    rv += [ '  ' + l + r'\\' for l in lines]
    rv += [r'\end{pmatrix}']
    return '\n'.join(rv)


In [32]:
C = sp.Matrix([[1, 3, 5, 7, 2],
              [sp.Rational(1,3), 1, 3, 3, sp.Rational(1,5)],
              [sp.Rational(1,5), sp.Rational(1,3), 1, 2, sp.Rational(1,4)],
              [sp.Rational(1,7), sp.Rational(1,3), sp.Rational(1,2), 1, sp.Rational(1,3)],
              [sp.Rational(1,2), 5, 4, 3, 1]])

A_1 = sp.Matrix([[1, 2, 3, 7, 4],
              [sp.Rational(1,2), 1, 4, sp.Rational(1,4), sp.Rational(1,3)],
              [sp.Rational(1,3), sp.Rational(1,4), 1, sp.Rational(1,5), sp.Rational(1,2)],
              [sp.Rational(1,7), sp.Rational(1,4), 5, 1, sp.Rational(1,3)],
              [sp.Rational(1,4), 3, 2, 3, 1]])

A_2 = sp.Matrix([[1, 2, 3, 6, 4],
              [sp.Rational(1,3), 1, 6, 6, 6],
              [sp.Rational(1,5), sp.Rational(1,6), 1, 3, 4],
              [sp.Rational(1,6), sp.Rational(1,6), sp.Rational(1,3), 1, 3],
              [sp.Rational(1,4), sp.Rational(1,6), sp.Rational(1,4), sp.Rational(1,3), 1]])

A_3 = sp.Matrix([[1, 2, 3, sp.Rational(1,2), sp.Rational(1,5)],
              [sp.Rational(1,2), 1, 2, sp.Rational(1,4), sp.Rational(1,4)],
              [sp.Rational(1,3), sp.Rational(1,2), 1, sp.Rational(1,6), sp.Rational(1,6)],
              [2, 4, 6, 1, sp.Rational(1,2)],
              [5, 4, 6, 2, 1]])

A_4 = sp.Matrix([[1, 3, 5, 7, 9],
              [sp.Rational(1,3), 1, 2, 5, 5],
              [sp.Rational(1,5), sp.Rational(1,3), 1, 3, 3],
              [sp.Rational(1,7), sp.Rational(1,5), sp.Rational(1,3), 1, 2],
              [sp.Rational(1,9), sp.Rational(1,5), sp.Rational(1,3), sp.Rational(1,2), 1]])

A_5 = sp.Matrix([[1, 2, 3, 2, 3],
              [sp.Rational(1,2), 1, 3, 4, 5],
              [sp.Rational(1,3), sp.Rational(1,3), 1, 2, 2],
              [sp.Rational(1,3), sp.Rational(1,4), sp.Rational(1,2), 1, 3],
              [sp.Rational(1,2), sp.Rational(1,5), sp.Rational(1,2), sp.Rational(1,3), 1]])

x = np.array([1,1,1,1,1])
epsilon = 1e-9

In [110]:
matrices = [C, A_1, A_2, A_3, A_4, A_5]

# print(iterative_matrix_vector_multiply_normalized(C, x, epsilon))
results = apply_to_multiple_matrices_numpy_corrected(matrices, x, epsilon)
print(results)
vectors=[]
for res in results:
    # print("A: ", res[0])
    # print("Lambda: ", res[1])
    # print("Iterations: ", res[2])
    # print("Latex: ", pmatrix(res[0][:, np.newaxis], 4))
    vectors.append(res[0])

# print(results[0][0]*results[0][1], np.dot(C,results[0][0]))

Cvec = vectors.pop(0)
print(vectors)
print(vectors[1][2]*Cvec[1])
res = vectors*Cvec[:, np.newaxis]
# print("\n", Cvec, "\n", vectors, "\n", res)
print(pmatrix(sum(res)[:,np.newaxis], precision=4))
sumres = sum(res)
print(pmatrix((sumres/max(sumres))[:,np.newaxis], precision=4))

NameError: name 'iterative_matrix_vector_multiply_normalized' is not defined

: 

In [390]:
sp_rad = spectral_radius_symbolic(C, {})
sp_rad

10**(1/4)

In [341]:
sp_rad_sym = sp.Symbol('lambda')
D = klini_symbolic(C, {sp_rad_sym: sp_rad})
D
#sp.latex(D)

Matrix([
[              1,     10/lambda**2, 30/lambda**3, 60/lambda**4,        2/lambda],
[3/(5*lambda**2),                1,     3/lambda,  6/lambda**2,     2/lambda**3],
[   1/(5*lambda), 10/(3*lambda**3),            1,     2/lambda, 2/(3*lambda**2)],
[   lambda**(-4),  5/(3*lambda**2),  5/lambda**3,            1,    1/(3*lambda)],
[    3/lambda**3,         5/lambda, 15/lambda**2, 30/lambda**3,               1]])

In [175]:
def invert_max_columns(matrix, sym_val_dict):
    if not isinstance(matrix, sp.Matrix) or not isinstance(sym_val_dict, dict):
        return "Error: Invalid input"

    rows, cols = matrix.shape
    inverted_maxima = []

    for j in range(cols):
        column = matrix[:, j]
        try:
            num_column = np.array([expr.subs(sym_val_dict) for expr in column])
            max_index = np.argmax(num_column)
            max_element = column[max_index]
            inverted_max = 1 / max_element
            inverted_maxima.append(inverted_max)
        except (TypeError, ValueError, ZeroDivisionError):
            return f"Error in column {j + 1}"

    return sp.Matrix(inverted_maxima).T

Matrix([[1/5, 1/z, 1/6]])
Matrix([[1, 1/5, 1/5, 1/7, 1/2]])


In [253]:
S = invert_max_columns(D, {sp_rad_sym: sp_rad})
S


Matrix([[1, lambda**2/10, lambda**3/30, lambda**4/60, lambda/2]])

In [259]:
def inverse_min_columns_symbolic(matrix, sym_val_dict):
    if not isinstance(matrix, sp.Matrix) or not isinstance(sym_val_dict, dict):
        return "Error: Invalid input"

    rows, cols = matrix.shape
    minima = []

    for j in range(cols):
        column = matrix[:, j]
        try:
            num_column = np.array([expr.subs(sym_val_dict) for expr in column])
            min_index = np.argmin(num_column)
            min_element = column[min_index]
            min_element = 1 / min_element
            minima.append(min_element)
        except (TypeError, ValueError):
            return f"Error in column {j + 1}"

    return sp.Matrix(minima).T

In [260]:
res_min = inverse_min_columns_symbolic(D, {sp_rad_sym: sp_rad})
res_min

Matrix([[lambda**4, 3*lambda**2/5, lambda**3/5, 1, 3*lambda]])

In [261]:
w_1 = sp.Matrix([1,
                 1/sp_rad_sym**2,
                 sp.Rational(1,3)/sp_rad_sym,
                 sp.Rational(1,6),
                 sp_rad_sym*sp.Rational(1,2)])
w_1

Matrix([
[           1],
[lambda**(-2)],
[1/(3*lambda)],
[         1/6],
[    lambda/2]])

In [262]:
w_2 = sp.Matrix([1, 
                 sp.Rational(3,5)/sp_rad_sym**2,
                 sp.Rational(1,5)/sp_rad_sym,
                 sp.Rational(1,10),
                 3/sp_rad_sym**3])
w_2

Matrix([
[              1],
[3/(5*lambda**2)],
[   1/(5*lambda)],
[           1/10],
[    3/lambda**3]])

In [288]:
A = [A_1, A_2, A_3, A_4, A_5]
B_1 = max_algebra_addition_symbolic([w_1[i]*A[i] for i in range(5)], {sp_rad_sym: sp_rad})
B_1

Matrix([
[           1,            2, 3,        7,          4],
[         1/2,            1, 4, 2*lambda, 5*lambda/2],
[         1/3,     lambda/6, 1,   lambda,     lambda],
[2/(3*lambda), 4/(3*lambda), 5,        1, 3*lambda/2],
[5/(3*lambda),            3, 2,        3,          1]])

In [289]:
sp_rad1 = spectral_radius_symbolic(B_1, {sp_rad_sym: sp_rad})
sp_rad1

sqrt(30)*sqrt(lambda)/2

In [334]:
sp_rad_sym1 = sp.Symbol('mu_1')
S = klini_symbolic(B_1, {sp_rad_sym: sp_rad, sp_rad_sym1: sp_rad1.subs({sp_rad_sym: sp_rad})})
S.subs({sp_rad_sym: 2*sp_rad_sym1**2/15})


Matrix([
[             1, 21/(5*mu_1), 35/mu_1**2,    7/mu_1,       7/5],
[25/(6*mu_1**2),           1,     5/mu_1,         1,    mu_1/3],
[ 5/(3*mu_1**2),         2/5,          1, 2*mu_1/15, 2*mu_1/15],
[ 5/(2*mu_1**2),         3/5,     5/mu_1,         1,    mu_1/5],
[25/(2*mu_1**3),      3/mu_1, 15/mu_1**2,    3/mu_1,         1]])

In [330]:
S_res = invert_max_columns(S, {sp_rad_sym: sp_rad, sp_rad_sym1: sp_rad1.subs({sp_rad_sym: sp_rad})})
S_res

Matrix([[1, 2*mu_1**3/(63*lambda), mu_1**2/35, mu_1/7, 2*mu_1**2/(21*lambda)]])

In [342]:
B_2 = max_algebra_addition_symbolic([w_2[i]*A[i] for i in range(5)], {sp_rad_sym: sp_rad})
B_2
#sp.latex(B_2)

Matrix([
[           1,            2, 3,            7,            4],
[         1/2,            1, 4, 12/lambda**3, 15/lambda**3],
[         1/3,          1/4, 1,  6/lambda**3,  6/lambda**3],
[2/(5*lambda), 4/(5*lambda), 5,            1,  9/lambda**3],
[    1/lambda,            3, 2,            3,            1]])

In [343]:
sp_rad2 = spectral_radius_symbolic(B_2, {sp_rad_sym: sp_rad})
sp_rad2

3*sqrt(5)*sqrt(lambda**(-3))

In [353]:
sp_rad_sym2 = sp.Symbol("mu_2")
sp_rad
sp_rad2
Q = klini_symbolic(B_2, {sp_rad_sym: sp_rad, sp_rad_sym2: sp_rad2.subs({sp_rad_sym:sp_rad})})
Q.subs({sp_rad_sym: (45/sp_rad_sym2**2)**(sp.Rational(1,3))})

Matrix([
[            1, 14/mu_2**2, 35/mu_2**2, 7/mu_2, 14/(3*mu_2)],
[5/(3*mu_2**2),          1,     5/mu_2,      1,      mu_2/3],
[   1/(3*mu_2),        2/5,          1,    2/5,   2*mu_2/15],
[5/(3*mu_2**2),     2/mu_2,     5/mu_2,      1,         2/3],
[    5/mu_2**3,     3/mu_2, 15/mu_2**2, 3/mu_2,           1]])

In [356]:
Q_min = inverse_min_columns_symbolic(Q, {sp_rad_sym: sp_rad, sp_rad_sym2: sp_rad2.subs({sp_rad_sym: sp_rad})})
Q_min.subs({sp_rad_sym: (45/sp_rad_sym2**2)**(sp.Rational(1,3))})

Matrix([[3*mu_2, 5/2, 1, 5/2, 15/(2*mu_2)]])

In [358]:
Q_max = invert_max_columns(Q, {sp_rad_sym: sp_rad, sp_rad_sym2: sp_rad2.subs({sp_rad_sym: sp_rad})})
Q_max.subs({sp_rad_sym: (45/sp_rad_sym2**2)**(sp.Rational(1,3))})

Matrix([[1, mu_2**2/14, mu_2**2/35, mu_2/7, 3*mu_2/14]])

In [360]:
elwise = lambda a, b, f: [f(i,j) for i,j in zip(a,b)]
div = lambda a, b: elwise(a,b, lambda x,y: x/y)

In [366]:
div(Q_min.subs({sp_rad_sym: sp_rad, sp_rad_sym2: sp_rad2.subs({sp_rad_sym: sp_rad})}), Q_max.subs({sp_rad_sym: sp_rad, sp_rad_sym2: sp_rad2.subs({sp_rad_sym: sp_rad})}))

[9*2**(5/8)*5**(1/8)/2,
 7*10**(3/4)/9,
 7*10**(3/4)/9,
 7*2**(3/8)*5**(7/8)/6,
 7*10**(3/4)/9]

In [372]:
v = sp.Matrix([1,
               sp.Rational(5, 3)/sp_rad_sym2**2,
               sp.Rational(1,3)/sp_rad_sym2,
               sp.Rational(5,3)/sp_rad_sym2**2,
               5/sp_rad_sym2**3]
               )
v.subs({sp_rad_sym2: sp_rad2.subs({sp_rad_sym: sp_rad})}).evalf(4)

Matrix([
[   1.0],
[0.2083],
[0.1178],
[0.2083],
[0.2209]])

In [382]:
print(1, 4/5, 2/np.sqrt(10))

1 0.8 0.6324555320336759


In [383]:
print(1, 1, np.sqrt(10)/4)

1 1 0.7905694150420949


In [398]:
35/sp_rad2.subs({sp_rad_sym: sp_rad}).evalf(4)/2

6.186